In [1]:
import json
import os
import pandas as pd
from datetime import datetime, timedelta
from time import sleep
from itertools import combinations
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

In [2]:
# # Comman function for init driver
# def init_driver(url):
#     driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH)
#     driver.maximize_window()
#     driver.get('https://paytm.com/flights')
#     driver.get(url)
#     return driver

## https://stackoverflow.com/questions/53039551/selenium-webdriver-modifying-navigator-webdriver-flag-to-prevent-selenium-detec
def init_driver(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    try:
        driver.get(url)
    except:
        print('error 4')
            
    return driver


def wait_for_page_load(xpath_url):
    global driver
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
        return 1
    except TimeoutException:
        
        print("Timed out waiting for page to load")
        try:
            all_text = driver.find_element_by_xpath('//div[@class="_2jvV"]').text
            if 'No flights found' in all_text:
                return 2
            
            driver.close()
        except:
            driver = init_driver(url)
            print('driver not found')
            
        # sleep(2)
        
        return False
    except NoSuchElementException:
        print("Element not found")
        return False


In [3]:
print('Starting...')
url = 'https://paytm.com/flights'
CHROMEDRIVER_PATH = './chromedriver'
WINDOW_SIZE = "1920,1080"
chrome_options = Options()
# chrome_options.headless = True
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
# chrome_options.add_argument('--user-data-dir=./User_Data')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--remote-debugging-port=9222")
# chrome_options.add_experimental_option("excludeSwitches",["ignore-certificate-errors"])

print('initiated...')

preferences = {
	# "profile.default_content_settings.popups": 0,
	"download.default_directory": os.getcwd() + os.path.sep,
	# "directory_upgrade": True
}

chrome_options.add_experimental_option('prefs', preferences)

# chrome_options.binary_location = CHROME_PATH

driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, options=chrome_options)
driver.maximize_window()


driver.get(url)



Starting...
initiated...


In [4]:
#1500

In [5]:
pd.options.display.max_rows = 100

In [6]:
# all city list
citys = [
    'DEL-Delhi',
    'BOM-Mumbai',
    'BLR-Bengaluru',
    'HYD-Hyderabad',
    'CU-Kolkata',
    'MAA-Chennai',
    'AMD-Ahmedabad',
    'GOI-Goa',
    'PAT-Patna',
    'COK-Kochi',
    'LKO-Lucknow',
    'GAU-Guwahati',
    'PNQ-Pune',
    'JAI-Jaipur',
    'SXR-Srinagar',
    'BBI-Bhubaneshwar',
    'IXB-Bagdogra',
    'VNS-Varanasi',
    'IXC-Chandigarh',
    'IXR-Ranchi',
    'VTZ-Vishakhapatnam',
    'RPR-Raipur',
    'NAG-Nagpur',
    'TRV-Thiruvananthapuram',
    'CCJ-Kozhikode',
    'IDR-Indore',
    'ATQ-Amritsar',
    'IXJ-Jammu',
    'CJB-Coimbatore',
    'DED-Dehradun',
]

In [7]:
possible_combinations = list(combinations(citys,2))

In [8]:
def collect_data(driver, today, next_date, day_before, source, destination):
    
    status_1 = wait_for_page_load('//div[@class="_2JLq"]/div[@class="_2TFk"]')
    status_2 = wait_for_page_load('//div[@class="_3H-S _1Eia"]')
    
    if status_1 == 2 and status_2 == 2:
        print('flight not found')
        return
    
    all_boxes = driver.find_elements_by_xpath('//div[@class="_2JLq"]/div[@class="_2TFk"]')
    try:
        for one_box in all_boxes:
            try:
                flight_name = one_box.find_element_by_xpath('.//div[@class="_3H-S _1Eia"]').text
                flight_id = one_box.find_element_by_xpath('.//div[@class="NqXj _2GoO"]').text
                start_time = one_box.find_element_by_xpath('.//div[@class="_3H-S"]').text
                end_time = one_box.find_element_by_xpath('.//div[@class="_3H-S _1wD5"]').text.split('\n')[0]
                travel_time = one_box.find_element_by_xpath('.//div[@class="vY4t"]').text
                stops = one_box.find_element_by_xpath('.//div[@class="_7BOG"]').text
                fare = one_box.find_element_by_xpath('.//div[@class="_1cxG"]').text

                all_data.append({
                    'flight_name': flight_name,
                    'flight_id': flight_id,
                    'start_time': start_time,
                    'end_time': end_time,
                    'travel_time': travel_time,
                    'stops': stops,
                    'fare': fare,
#                     'today': today,
#                     'next_date': next_date,
#                     'day_before': day_before
                    'source': source,
                    'destination': destination
                })
            except Exception as e:
                print(e)
                print('error2')
                # sleep(5)
            
    except Exception as e:
        print(e)
        print('error3')
        # sleep(5)

In [9]:
def search_page(driver, source, destination):
    
    try:
        clear_inputs = driver.find_elements_by_xpath('//div[@class="fl-input-clear"]')
        clear_inputs[0].click()
        clear_inputs[1].click()
    except:
        print('cleared already')
        
    sleep(2)
    wait_for_page_load('//div[@class="_31PN"]//input')
    
    inputs = driver.find_elements_by_xpath('//div[@class="_31PN"]//input')

    start_input = inputs[0]
    end_input = inputs[1]

    start_input.clear()
    start_input.send_keys(source)
    sleep(2)
    wait_for_page_load('//div[@class="_2Alu"]//div[@class="_2xgL"]')
    # Select First suggestion
    driver.find_element_by_xpath('//div[@class="_2Alu"]//div[@class="_2xgL"]').click()

    end_input.clear()
    end_input.send_keys(destination)
    sleep(2)
    
    wait_for_page_load('//div[@class="_2Alu"]//div[@class="_2xgL"]')
    # Select First suggestion
    driver.find_element_by_xpath('//div[@class="_2Alu"]//div[@class="_2xgL"]').click()

    driver.find_element_by_xpath("//button[text()='Search']").click()
    sleep(2)
    

In [10]:
all_data = []
for i, one_combination in enumerate(possible_combinations):
    source, destination = one_combination
    today = datetime.today() + timedelta(5)
#     for i in range(1, 11):
    try:
        next_date = today.strftime('%Y-%m-%d') # (today + timedelta(i)).strftime('%Y-%m-%d')
        # url = f'https://paytm.com/flights/flightSearch/{source}/{destination}/1/0/0/E/{next_date}'
        # driver.get(url)
        search_page(driver, source, destination)
        sleep(2)
        collect_data(driver, today, next_date, 0, source, destination)
    except Exception as e:
        # sleep(2)
        print(e)
        print('error1')
        try:
            driver.close()
        except:
            print('driver not found')
            pd.DataFrame(all_data).to_csv('flight_data__.csv')
            
        driver = init_driver(url)
        
    print(one_combination, i+1, len(possible_combinations), (i+1)/ len(possible_combinations))
df = pd.DataFrame(all_data)
df

cleared already
('DEL-Delhi', 'BOM-Mumbai') 1 435 0.0022988505747126436
('DEL-Delhi', 'BLR-Bengaluru') 2 435 0.004597701149425287
('DEL-Delhi', 'HYD-Hyderabad') 3 435 0.006896551724137931
('DEL-Delhi', 'CU-Kolkata') 4 435 0.009195402298850575
('DEL-Delhi', 'MAA-Chennai') 5 435 0.011494252873563218
('DEL-Delhi', 'AMD-Ahmedabad') 6 435 0.013793103448275862
('DEL-Delhi', 'GOI-Goa') 7 435 0.016091954022988506
('DEL-Delhi', 'PAT-Patna') 8 435 0.01839080459770115
('DEL-Delhi', 'COK-Kochi') 9 435 0.020689655172413793
('DEL-Delhi', 'LKO-Lucknow') 10 435 0.022988505747126436
('DEL-Delhi', 'GAU-Guwahati') 11 435 0.02528735632183908
('DEL-Delhi', 'PNQ-Pune') 12 435 0.027586206896551724
('DEL-Delhi', 'JAI-Jaipur') 13 435 0.029885057471264367
('DEL-Delhi', 'SXR-Srinagar') 14 435 0.03218390804597701
('DEL-Delhi', 'BBI-Bhubaneshwar') 15 435 0.034482758620689655
('DEL-Delhi', 'IXB-Bagdogra') 16 435 0.0367816091954023
('DEL-Delhi', 'VNS-Varanasi') 17 435 0.03908045977011494
('DEL-Delhi', 'IXC-Chandigar

('MAA-Chennai', 'GOI-Goa') 137 435 0.31494252873563217
('MAA-Chennai', 'PAT-Patna') 138 435 0.31724137931034485
('MAA-Chennai', 'COK-Kochi') 139 435 0.3195402298850575
('MAA-Chennai', 'LKO-Lucknow') 140 435 0.3218390804597701
('MAA-Chennai', 'GAU-Guwahati') 141 435 0.32413793103448274
('MAA-Chennai', 'PNQ-Pune') 142 435 0.3264367816091954
('MAA-Chennai', 'JAI-Jaipur') 143 435 0.32873563218390806
('MAA-Chennai', 'SXR-Srinagar') 144 435 0.3310344827586207
('MAA-Chennai', 'BBI-Bhubaneshwar') 145 435 0.3333333333333333
('MAA-Chennai', 'IXB-Bagdogra') 146 435 0.335632183908046
('MAA-Chennai', 'VNS-Varanasi') 147 435 0.33793103448275863
('MAA-Chennai', 'IXC-Chandigarh') 148 435 0.34022988505747126
('MAA-Chennai', 'IXR-Ranchi') 149 435 0.3425287356321839
('MAA-Chennai', 'VTZ-Vishakhapatnam') 150 435 0.3448275862068966
('MAA-Chennai', 'RPR-Raipur') 151 435 0.3471264367816092
('MAA-Chennai', 'NAG-Nagpur') 152 435 0.34942528735632183
('MAA-Chennai', 'TRV-Thiruvananthapuram') 153 435 0.3517241379

('GAU-Guwahati', 'SXR-Srinagar') 267 435 0.6137931034482759
('GAU-Guwahati', 'BBI-Bhubaneshwar') 268 435 0.6160919540229886
('GAU-Guwahati', 'IXB-Bagdogra') 269 435 0.6183908045977011
('GAU-Guwahati', 'VNS-Varanasi') 270 435 0.6206896551724138
('GAU-Guwahati', 'IXC-Chandigarh') 271 435 0.6229885057471264
('GAU-Guwahati', 'IXR-Ranchi') 272 435 0.6252873563218391
('GAU-Guwahati', 'VTZ-Vishakhapatnam') 273 435 0.6275862068965518
('GAU-Guwahati', 'RPR-Raipur') 274 435 0.6298850574712643
('GAU-Guwahati', 'NAG-Nagpur') 275 435 0.632183908045977
('GAU-Guwahati', 'TRV-Thiruvananthapuram') 276 435 0.6344827586206897
('GAU-Guwahati', 'CCJ-Kozhikode') 277 435 0.6367816091954023
('GAU-Guwahati', 'IDR-Indore') 278 435 0.639080459770115
('GAU-Guwahati', 'ATQ-Amritsar') 279 435 0.6413793103448275
('GAU-Guwahati', 'IXJ-Jammu') 280 435 0.6436781609195402
('GAU-Guwahati', 'CJB-Coimbatore') 281 435 0.6459770114942529
('GAU-Guwahati', 'DED-Dehradun') 282 435 0.6482758620689655
('PNQ-Pune', 'JAI-Jaipur') 2

('IXR-Ranchi', 'IDR-Indore') 386 435 0.8873563218390804
Timed out waiting for page to load
Timed out waiting for page to load
flight not found
('IXR-Ranchi', 'ATQ-Amritsar') 387 435 0.8896551724137931
Timed out waiting for page to load
Timed out waiting for page to load
flight not found
('IXR-Ranchi', 'IXJ-Jammu') 388 435 0.8919540229885058
('IXR-Ranchi', 'CJB-Coimbatore') 389 435 0.8942528735632184
Timed out waiting for page to load
Timed out waiting for page to load
flight not found
('IXR-Ranchi', 'DED-Dehradun') 390 435 0.896551724137931
Timed out waiting for page to load
Timed out waiting for page to load
flight not found
('VTZ-Vishakhapatnam', 'RPR-Raipur') 391 435 0.8988505747126436
Timed out waiting for page to load
Timed out waiting for page to load
flight not found
('VTZ-Vishakhapatnam', 'NAG-Nagpur') 392 435 0.9011494252873563
('VTZ-Vishakhapatnam', 'TRV-Thiruvananthapuram') 393 435 0.903448275862069
('VTZ-Vishakhapatnam', 'CCJ-Kozhikode') 394 435 0.9057471264367816
('VTZ-Vis

,flight_name,flight_id,start_time,end_time,travel_time,stops,fare,source,destination
0,Go First,G8 - 336,14:20,16:35,2h 15m,Non Stop,"5,954",DEL-Delhi,BOM-Mumbai
1,Go First,G8 - 392,12:45,15:00,2h 15m,Non Stop,"5,954",DEL-Delhi,BOM-Mumbai
2,Go First,G8 - 323,18:05,20:25,2h 20m,Non Stop,"5,954",DEL-Delhi,BOM-Mumbai
3,Go First,G8 - 717,16:45,21:25,4h 40m,1 stop at Ahmedabad,"5,954",DEL-Delhi,BOM-Mumbai
4,Go First,G8 - 167,15:00,21:55,6h 55m,1 stop at Ranchi,"5,954",DEL-Delhi,BOM-Mumbai
...,...,...,...,...,...,...,...,...,...
7029,IndiGo,6E - 137,11:55,23:15,11h 20m,"2 Stops (SXR, DEL)","12,092",IXJ-Jammu,CJB-Coimbatore
7030,IndiGo,6E - 124,16:30,23:15,6h 45m,1 stop at Delhi,"14,454",IXJ-Jammu,CJB-Coimbatore
7031,IndiGo,6E - 6738,13:10,23:15,10h 5m,1 stop at Delhi,"14,454",IXJ-Jammu,CJB-Coimbatore
7032,IndiGo,6E - 6152,06:05,17:05,11h,1 stop at Mumbai,"10,782",CJB-Coimbatore,DED-Dehradun


In [12]:
df.to_csv('flight_data___.csv')